# ETL 'DF_ITEMS'


CONVOCO LIBRERIAS

In [1]:
# Realizo carga de librerias.

import ast
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

CARGO EL ARCHIVO Y LO TRANSFORMO EN UN DATAFRAME.

In [2]:
filas = list()
with open(r"D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\ARCHIVOS JSON\australian_users_items.json", "r", encoding="Latin-1") as archivo:
    for linea in archivo.readlines():
        filas.append(ast.literal_eval(linea))

df_items = pd.DataFrame(filas)


In [3]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


ANALIZO EL FORMATO DEL DATAFRAME

In [4]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


ANALIZO EL TIPO DE DATOS QUE CONTIENEN SUS CAMPOS.

In [5]:
def verificar_columna(column):
    tiene_anidados = any(isinstance(valor, (list, dict, tuple,set)) for valor in column)
    tipos_datos = set(type(valor).__name__ for valor in column)
    return tiene_anidados, tipos_datos

# Verificar cada columna especificada
for col in df_items:
    tiene_anidados, tipos_datos = verificar_columna(df_items[col])
    if tiene_anidados:
        print(f'La columna "{col}" tiene datos anidados')
        print(f'Tipos de datos en la columna "{col}": {tipos_datos}')
    else:
        print(f'La columna "{col}" no tiene datos anidados')
        print(f'Tipos de datos en la columna "{col}": {tipos_datos}')

La columna "user_id" no tiene datos anidados
Tipos de datos en la columna "user_id": {'str'}
La columna "items_count" no tiene datos anidados
Tipos de datos en la columna "items_count": {'int'}
La columna "steam_id" no tiene datos anidados
Tipos de datos en la columna "steam_id": {'str'}
La columna "user_url" no tiene datos anidados
Tipos de datos en la columna "user_url": {'str'}
La columna "items" tiene datos anidados
Tipos de datos en la columna "items": {'list'}


In [6]:
df_items['user_id']

0              76561197970982479
1                        js41637
2                      evcentric
3                     Riot-Punch
4                          doctr
                  ...           
88305          76561198323066619
88306          76561198326700687
88307    XxLaughingJackClown77xX
88308          76561198329548331
88309          edward_tremethick
Name: user_id, Length: 88310, dtype: object

In [7]:
df_items['steam_id']

0        76561197970982479
1        76561198035864385
2        76561198007712555
3        76561197963445855
4        76561198002099482
               ...        
88305    76561198323066619
88306    76561198326700687
88307    76561198328759259
88308    76561198329548331
88309    76561198331598578
Name: steam_id, Length: 88310, dtype: object

PROCEDO A DESANIDAR VALORES.

In [9]:
itemsdf_expandido = df_items.explode('items')
itemsdf_desanidado = pd.json_normalize(itemsdf_expandido['items'])

itemsdf_expandido = itemsdf_expandido.reset_index(drop=True)
itemsdf_desanidado = itemsdf_desanidado.reset_index(drop=True)

df_items_final = pd.concat([itemsdf_expandido, itemsdf_desanidado], axis=1)
df_items_final.head(203)

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...
198,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '219740', 'item_name': 'Don't Star...",219740,Don't Starve,139.0,0.0
199,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '322330', 'item_name': 'Don't Star...",322330,Don't Starve Together,0.0,0.0
200,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '222730', 'item_name': 'Reus', 'pl...",222730,Reus,319.0,0.0
201,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '241600', 'item_name': 'Rogue Lega...",241600,Rogue Legacy,73.0,0.0


In [10]:
df_items_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   items             object 
 5   item_id           object 
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 355.0+ MB


ELIMINO COLUMNAS QUE NO HACEN AL ANALISIS.

In [11]:
#Se eliminan las columnas de itemsdf
df_items_final.drop(['items', 'playtime_2weeks', 'user_url','item_name'], axis=1, inplace=True)
df_items_final

,user_id,items_count,steam_id,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,10,6.0
1,76561197970982479,277,76561197970982479,20,0.0
2,76561197970982479,277,76561197970982479,30,7.0
3,76561197970982479,277,76561197970982479,40,0.0
4,76561197970982479,277,76561197970982479,50,0.0
...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,373330,0.0
5170011,76561198329548331,7,76561198329548331,388490,3.0
5170012,76561198329548331,7,76561198329548331,521570,4.0
5170013,76561198329548331,7,76561198329548331,519140,3.0


ANALIZO Y BUSCO LOS NULOS

In [12]:
# Mostrar los nombres de las columnas en df_items
print(df_items_final.columns)

# Verificar si 'item_id' existe en el DataFrame
if 'item_id' in df_items_final.columns:
    # Contar el número de valores NaN en la columna 'item_id'
    num_nan_item_id = df_items_final['item_id'].isna().sum()
    
    # Calcular el total de filas en el DataFrame
    total_rows = len(df_items_final)
    
    # Calcular el porcentaje de valores NaN
    percentage_nan_item_id = (num_nan_item_id / total_rows) * 100
    
    # Imprimir el número y el porcentaje de valores NaN
    print(f"Número de NaN en la columna 'item_id': {num_nan_item_id}")
    print(f"Porcentaje de NaN en la columna 'item_id': {percentage_nan_item_id:.2f}%")
    print(f"Total de registros en el DataFrame: {total_rows}")
else:
    print("La columna 'item_id' no existe en el DataFrame.")



Index(['user_id', 'items_count', 'steam_id', 'item_id', 'playtime_forever'], dtype='object')
Número de NaN en la columna 'item_id': 16806
Porcentaje de NaN en la columna 'item_id': 0.33%
Total de registros en el DataFrame: 5170015


PROCEDO A ELIMINAR LOS NULOS.

In [13]:
df_items_final = df_items_final.dropna(how='all')
df_items_final

,user_id,items_count,steam_id,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,10,6.0
1,76561197970982479,277,76561197970982479,20,0.0
2,76561197970982479,277,76561197970982479,30,7.0
3,76561197970982479,277,76561197970982479,40,0.0
4,76561197970982479,277,76561197970982479,50,0.0
...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,373330,0.0
5170011,76561198329548331,7,76561198329548331,388490,3.0
5170012,76561198329548331,7,76561198329548331,521570,4.0
5170013,76561198329548331,7,76561198329548331,519140,3.0


In [14]:
# Eliminar filas con NaN en la columna 'item_id'
df_items_final.dropna(subset=['item_id'], inplace=True)

# Verificar el resultado
print(df_items)

                       user_id  items_count           steam_id  \
0            76561197970982479          277  76561197970982479   
1                      js41637          888  76561198035864385   
2                    evcentric          137  76561198007712555   
3                   Riot-Punch          328  76561197963445855   
4                        doctr          541  76561198002099482   
...                        ...          ...                ...   
88305        76561198323066619           22  76561198323066619   
88306        76561198326700687          177  76561198326700687   
88307  XxLaughingJackClown77xX            0  76561198328759259   
88308        76561198329548331            7  76561198329548331   
88309        edward_tremethick            0  76561198331598578   

                                                user_url  \
0      http://steamcommunity.com/profiles/76561197970...   
1                   http://steamcommunity.com/id/js41637   
2                 http://st

ELIMINO DUPLICADOS.

In [15]:
df_items_final.drop_duplicates(subset=None, keep='first', inplace=False)
df_items_final

,user_id,items_count,steam_id,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,10,6.0
1,76561197970982479,277,76561197970982479,20,0.0
2,76561197970982479,277,76561197970982479,30,7.0
3,76561197970982479,277,76561197970982479,40,0.0
4,76561197970982479,277,76561197970982479,50,0.0
...,...,...,...,...,...
5170009,76561198329548331,7,76561198329548331,346330,0.0
5170010,76561198329548331,7,76561198329548331,373330,0.0
5170011,76561198329548331,7,76561198329548331,388490,3.0
5170012,76561198329548331,7,76561198329548331,521570,4.0


RESETEO INDICES

In [16]:
df_items_final.reset_index(drop=True, inplace=True)

GUARDO EL DATAFRAME EN FORMATO 'CSV' Y 'PARQUET'

In [17]:
df_items_final.to_csv(r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\df_items.csv', index=False)

In [18]:
df_items_final.to_parquet(r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\DATAFRAMES PARKET\df_items.parquet')

c:\Users\Arnaldo\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
